In [7]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers
# 🔒 Reproducibility setup
import random
import os
from tqdm import tqdm 
tf.config.run_functions_eagerly(True)

import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [8]:
SEED = 42  # Choose any number you want, but be consistent
# Set seeds
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# (Optional) Force single-threaded operations
os.environ['TF_DETERMINISTIC_OPS'] = '1'  # Try to make TF ops deterministic
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

In [9]:
#If GPU is available 
# Step 1: Set GPU environment (should still be early, before TF uses GPU)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Step 2: GPU setup for Jupyter (safe inside notebook)
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU memory growth enabled.")
    except RuntimeError as e:
        print("⚠️ RuntimeError during GPU setup:", e)
else:
    print("❌ No GPU found.")

# Step 3: Show detected GPUs
print("Available GPU(s):", tf.config.list_logical_devices("GPU"))


In [10]:

# In a new cell at the top of your notebook
parameters = dict()

# --- Model Architecture ---
parameters['module'] = 'gru'
parameters['hidden_dim'] = 64
parameters['num_layers'] = 3
parameters['z_dim'] = 64
parameters['seq_len'] = 24      #This is the output_dim for the Embedder     ??????????????? always
parameters['dim'] = 14    # This is the output_dim for the Recovery (your original feature count)   ???????????????

# --- Training Hyperparameters ---
parameters['iterations'] = 4000
parameters['batch_size'] = 128
parameters['learning_rate'] = 0.001
parameters['embedder_iterations'] = 4000
parameters['supervisor_iterations'] = 5000

# --- Loss Weights ---
parameters['gamma'] = 10
parameters['eta'] = 1
parameters['lambda'] = 10
#########################

In [11]:

def make_dataset(data, batch_size):
    ds = tf.data.Dataset.from_tensor_slices(data.astype('float32'))
    ds = ds.shuffle(buffer_size=min(len(data), 10000), reshuffle_each_iteration=True)
    ds = ds.repeat()                               # infinite
    ds = ds.batch(batch_size, drop_remainder=True)
    ds = ds.prefetch(1)                            # small, predictable prefetch
    return ds

In [12]:

class Embedder(tf.keras.Model):
    def __init__(self, hidden_units, num_layers, output_dim, module_name):
        super(Embedder, self).__init__()
        
        # Determine which RNN cell to use based on the module_name
        if module_name == 'gru':
            rnn_cell = layers.GRU
        elif module_name == 'lstm':
            rnn_cell = layers.LSTM
        elif module_name == 'lstmLN':
            rnn_cell = layers.lstmLN
            
        else:
            raise ValueError(f"Unknown RNN module: {module_name}")
            
        self.rnn_layers = [
            rnn_cell(hidden_units, return_sequences=True) for _ in range(num_layers)
        ]
        self.dense = layers.Dense(output_dim, activation='sigmoid')

    def call(self, x, training=True):
        # Pass the input through the stack of RNN layers
        for layer in self.rnn_layers:
            x = layer(x, training=training)
        return self.dense(x)


In [13]:
class Recovery(tf.keras.Model):
    def __init__(self, hidden_units, num_layers, output_dim, module_name):
        super(Recovery, self).__init__()

        # Determine which RNN cell to use based on the module_name
        if module_name == 'gru':
            rnn_cell = layers.GRU
        elif module_name == 'lstm':
            rnn_cell = layers.LSTM
        elif module_name == 'lstmLN':
            rnn_cell = layers.lstmLN
        else:
            raise ValueError(f"Unknown RNN module: {module_name}")

        self.rnn_layers = [
            rnn_cell(hidden_units, return_sequences=True) for _ in range(num_layers)
        ]
        self.dense = layers.Dense(output_dim, activation='sigmoid')

    def call(self, h, training=True):
        for layer in self.rnn_layers:
            h = layer(h, training=training)
        return self.dense(h)

In [14]:
class Generator(tf.keras.Model):
    def __init__(self, hidden_units, num_layers, output_dim, module_name):
        super(Generator, self).__init__()

        # Determine which RNN cell to use based on the module_name
        if module_name == 'gru':
            rnn_cell = layers.GRU
        elif module_name == 'lstm':
            rnn_cell = layers.LSTM
        elif module_name == 'lstmLN':
            rnn_cell = layers.lstmLN
        else:
            raise ValueError(f"Unknown RNN module: {module_name}")

        self.rnn_layers = [
            rnn_cell(hidden_units, return_sequences=True)
            for _ in range(num_layers)
        ]
        self.dense = layers.Dense(output_dim, activation='sigmoid')

    def call(self, x, training=True):
        for layer in self.rnn_layers:
            x = layer(x, training=training)
        return self.dense(x)

In [15]:
class Supervisor(tf.keras.Model):
    def __init__(self, hidden_units, num_layers, output_dim, module_name):
        super(Supervisor, self).__init__()

        # Determine which RNN cell to use based on the module_name
        if module_name == 'gru':
            rnn_cell = layers.GRU
        elif module_name == 'lstm':
            rnn_cell = layers.LSTM
        elif module_name == 'lstmLN':
            rnn_cell = layers.lstmLN
        else:
            raise ValueError(f"Unknown RNN module: {module_name}")

        self.rnn_layers = [
            rnn_cell(hidden_units, return_sequences=True)
            for _ in range(num_layers - 1)
        ]
        self.dense = layers.Dense(output_dim, activation='sigmoid')

    def call(self, x, training=True):
        for layer in self.rnn_layers:
            x = layer(x, training=training)
        return self.dense(x)

In [16]:
class Discriminator(tf.keras.Model):
    def __init__(self, hidden_units, num_layers, module_name='gru'):
        """
        Args:
            hidden_units: number of units per layer
            num_layers: number of layers
            module_name: 'gru' or 'lstm'
        """
        super(Discriminator, self).__init__()

        # Determine which RNN cell to use based on the module_name
        if module_name == 'gru':
            rnn_cell = layers.GRU
        elif module_name == 'lstm':
            rnn_cell = layers.LSTM
        elif module_name == 'lstmLN':
            rnn_cell = layers.lstmLN
        else:
            raise ValueError(f"Unknown RNN module: {module_name}")

        self.rnn_layers = [
            rnn_cell(hidden_units, return_sequences=True)
            for _ in range(num_layers)
        ]
        self.dense = layers.Dense(1, activation=None) # 1 score per time step

    def call(self, x, training=True):
        for layer in self.rnn_layers:
            x = layer(x, training=training)
        return self.dense(x)



In [18]:
mse = tf.keras.losses.MeanSquaredError()

# Assuming 'mse' is a function that calculates mean_squared_error
def embedding_loss(x, x_tilde, G_loss_S):
    """Calculates the final embedder loss as in the original code.
    
    Args:
      x: The original data batch.
      x_tilde: The reconstructed data batch.
      G_loss_S: The supervised loss value.
    """
    # 1. Calculate the base reconstruction loss (E_loss_T0)
   # E_loss_T0 = mse(x, x_tilde)

    # after (targets-only)
    E_loss_T0 = mse(x_batch[..., y_idx], x_tilde[..., y_idx])
    # 2. Apply the square root and the coefficient (E_loss0)
    E_loss0 = 10 * tf.sqrt(E_loss_T0)
    
    # 3. Add the supervised loss component to get the final loss (E_loss)
    E_loss = E_loss0 + 0.1 * G_loss_S
    
    return E_loss
    

In [19]:
def supervised_loss(H, H_hat_supervised):
    """Supervised loss = MSE over ALL dims (B, T-1, H)."""
    tgt = H[:, 1:, :]            # H at t+1
    pred = H_hat_supervised[:, :-1, :]  # S(H) at t
    return tf.reduce_mean(tf.square(tgt - pred))  # scalar

In [20]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True) # Set to True for raw output

def discriminator_loss(H, H_hat, E_hat, discriminator, gamma=1):
    """Adversarial loss for the discriminator, matching the provided reference.

    Args:
        H: The real latent sequences (from the embedder).
        H_hat: The fake latent sequences (from the supervisor).
        E_hat: The raw fake latent sequences (from the generator).
        gamma: The weight for the D_loss_fake_e term.
    """
    # 1. Get discriminator outputs
    Y_real = discriminator(H)
    Y_fake = discriminator(H_hat)
    Y_fake_e = discriminator(E_hat)

    # 2. Compute loss for each component using from_logits=True
    D_loss_real = bce(tf.ones_like(Y_real), Y_real)
    D_loss_fake = bce(tf.zeros_like(Y_fake), Y_fake)
    D_loss_fake_e = bce(tf.zeros_like(Y_fake_e), Y_fake_e)

    # 3. Combine the losses with the gamma parameter
    D_loss = D_loss_real + D_loss_fake + gamma * D_loss_fake_e

    return D_loss

In [21]:
embedder = Embedder(
    hidden_units=parameters['hidden_dim'],
    num_layers= parameters['num_layers'],
    output_dim=parameters['hidden_dim'],
    module_name=parameters['module']
    
)


recovery = Recovery(
    hidden_units=parameters['hidden_dim'],
    num_layers= parameters['num_layers'],
    output_dim=parameters['dim'],
    module_name=parameters['module']
)

supervisor = Supervisor(
    hidden_units=parameters['hidden_dim'],
    num_layers=parameters['num_layers'],
    output_dim=parameters['hidden_dim'],
    module_name=parameters['module']
)

generator = Generator(
    hidden_units=parameters['hidden_dim'],
    num_layers=parameters['num_layers'],
    output_dim=parameters['hidden_dim'],
    module_name=parameters['module']
)
discriminator = Discriminator(
    hidden_units=parameters['hidden_dim'],
    num_layers=parameters['num_layers'],
    module_name=parameters['module']

)


autoencoder_optimizer = tf.keras.optimizers.Adam(learning_rate=parameters['learning_rate'], beta_1=0.9)
supervisor_optimizer = tf.keras.optimizers.Adam(learning_rate=parameters['learning_rate'], beta_1=0.9)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=parameters['learning_rate'], beta_1=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=parameters['learning_rate'], beta_1=0.9 )


In [22]:

######################
def train_autoencoder_step(x_batch, y_idx):
    y_idx_tf = tf.constant(y_idx, dtype=tf.int32)  # [k]
    with tf.GradientTape() as tape:
        h = embedder(x_batch, training=True)
        x_tilde = recovery(h, training=True)

        x_y   = tf.gather(x_batch, y_idx_tf, axis=-1)   # (B,T,k)
        xt_y  = tf.gather(x_tilde, y_idx_tf, axis=-1)   # (B,T,k)
        e_loss_t0 = mse(x_y, xt_y)                      # targets-only

        loss = 10.0 * tf.sqrt(e_loss_t0 + 1e-8)

    vars_ = embedder.trainable_variables + recovery.trainable_variables
    grads = tape.gradient(loss, vars_)
    autoencoder_optimizer.apply_gradients(zip(grads, vars_))
    return loss



In [23]:
# Use the iteration count from your parameters
embedder_iterations = parameters['embedder_iterations']
BATCH_SIZE = parameters['batch_size'] 
# Build a fresh, repeating dataset + iterator for the EMBEDDER/AUTOENCODER phase
embedder_dataset = make_dataset(X_timegan, BATCH_SIZE)
embedder_batch_iter = iter(embedder_dataset)

print('Start Embedding Network Training')

e_loss_history = []
step = 0

# Train until we hit the target number of iterations
while step < embedder_iterations:
    x_batch = next(embedder_batch_iter)  # next reshuffled batch
    # Your AE step returns 10 * sqrt(MSE)
    step_e_loss0 = float(train_autoencoder_step(x_batch , y_idx))
    e_loss_history.append(step_e_loss0)

    if step % 1000 == 0:
        # Reference prints sqrt(MSE); you logged 10*sqrt(MSE) → divide by 10
        comparable_loss = np.mean(e_loss_history) / 10.0
        print(f'step: {step}/{embedder_iterations}, e_loss: {comparable_loss:.4f}')
        e_loss_history = []  # reset window

    step += 1

print('Finish Embedding Network Training')


NameError: name 'X_timegan' is not defined

In [ ]:
# After AE pretrain to control sup LR
embedder.trainable = False  # freeze E for supervisor warmup

from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
lr_sched = PiecewiseConstantDecay(boundaries=[2000], values=[1e-3, 3e-4])

# fresh optimizer for S (replaces the earlier constant-lr one)
supervisor_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_sched, beta_1=0.9)

In [ ]:
def train_supervisor_step(x_batch):
    with tf.GradientTape() as tape:
        H = embedder(x_batch, training=False)
        H = tf.stop_gradient(H)                 # keep E frozen

        H_in  = H[:, :-1, :]                    # input timesteps
        H_tgt = H[:,  1:, :]                    # target next-step

        H_pred = supervisor(H_in, training=True)  # shape (B, T-1, hidden_dim)
        loss = tf.reduce_mean(tf.square(H_tgt - H_pred))  # scalar MSE

    grads = tape.gradient(loss, supervisor.trainable_variables)
    supervisor_optimizer.apply_gradients(zip(grads, supervisor.trainable_variables))
    return loss


In [24]:

# ---- Supervisor-only training loop (repeat + RMSE logging) ----
supervisor_iterations =  parameters['supervisor_iterations']


# fresh dataset + iterator for Supervisor (don’t reuse AE iterator)
sup_ds = make_dataset(X_timegan, BATCH_SIZE)
sup_it = iter(sup_ds)

print("Start Training with Supervised Loss Only")
s_hist = []
for step in range(supervisor_iterations):  # e.g., 10_000
    x_batch = next(sup_it)
    step_mse = float(train_supervisor_step(x_batch))     # your S step (with stop_gradient)
    s_hist.append(step_mse)
    if step % 1000 == 0:
        print(f"step: {step}/{supervisor_iterations}, s_loss: {(np.mean(s_hist))**0.5:.4f}")
        s_hist = []
print("Finish Training with Supervised Loss Only")

NameError: name 'X_timegan' is not defined

In [ ]:

def moment_loss(x, x_hat, y_idx):
    # targets-only slice
    y_idx_tf = tf.constant(y_idx, tf.int32)
    x_y      = tf.gather(x,      y_idx_tf, axis=-1)   # (B,T,k)
    x_hat_y  = tf.gather(x_hat,  y_idx_tf, axis=-1)   # (B,T,k)

    # moments over batch
    mean_hat, var_hat = tf.nn.moments(x_hat_y, axes=[0])
    mean_x,   var_x   = tf.nn.moments(x_y,     axes=[0])

    std_hat = tf.sqrt(var_hat + 1e-6)
    std_x   = tf.sqrt(var_x   + 1e-6)

    G_loss_V1 = tf.reduce_mean(tf.abs(std_hat  - std_x))   # std diff
    G_loss_V2 = tf.reduce_mean(tf.abs(mean_hat - mean_x))  # mean diff
    return G_loss_V1 + G_loss_V2



In [ ]:

#######################################
print("🔄 Warm-up (Generator only; no adversarial)")

# One RNG for reproducible fresh noise each step
tf_gen = tf.random.Generator.from_seed(42)

# Fresh, repeating real-data iterator (don’t reuse AE iterator)
warmup_real_ds  = make_dataset(X_timegan.astype(np.float32), parameters['batch_size'])
warmup_real_it  = iter(warmup_real_ds)

# Freeze supervisor during G warm-up (optional but recommended)
supervisor.trainable = False

# Weights (typical TimeGAN-style scaling)
lambda_s = 100.0   # supervised on generated latents
lambda_v = 100.0   # two-moment loss in data space

warmup_steps = 500  # 200–1000 is usually enough
g_losses_Shat, g_losses_V, g_losses_total = [], [], []

for step in range(warmup_steps):
    real_batch = next(warmup_real_it)
    real_batch = tf.cast(real_batch, tf.float32)

    Z = sample_noise(parameters['batch_size'], parameters['seq_len'], parameters['z_dim'])

    with tf.GradientTape() as tape:
        # Latent path from noise
        E_hat  = generator(Z, training=True)               # [B, T, hidden]
        H_hat  = supervisor(E_hat, training=False)         # [B, T, hidden]  (S frozen)
        X_hat  = recovery(H_hat, training=True)            # [B, T, dim]

        # Supervised dynamics on generated latent (teacher forcing on itself)
        H_hat_in  = H_hat[:, :-1, :]
        H_hat_tgt = H_hat[:,  1:, :]
        H_hat_pred = supervisor(H_hat_in, training=False)  # S frozen
        G_loss_S_hat = tf.reduce_mean(tf.square(H_hat_tgt - H_hat_pred))

 

        G_loss_V = moment_loss(real_batch, X_hat, y_idx)

        # Total warm-up loss (no adversarial yet)
        G_total = lambda_s * G_loss_S_hat + lambda_v * G_loss_V


    # Update ONLY the generator
    g_vars = generator.trainable_variables
    g_grads = tape.gradient(G_total, g_vars)
    generator_optimizer.apply_gradients(zip(g_grads, g_vars))

    # Logging buffers
    g_losses_Shat.append(float(G_loss_S_hat))
    g_losses_V.append(float(G_loss_V))
    g_losses_total.append(float(G_total))

    if step % 100 == 0:
        print(f"Warmup {step}/{warmup_steps} | "
              f"G_S_hat: {np.mean(g_losses_Shat[-100:]):.6f} | "
              f"G_V: {np.mean(g_losses_V[-100:]):.6f} | "
              f"G_total: {np.mean(g_losses_total[-100:]):.6f}")

# (optional) unfreeze supervisor for joint training
supervisor.trainable = True


In [ ]:

#######################################
print("🔄 Warm-up (Generator only; no adversarial)")

# One RNG for reproducible fresh noise each step
tf_gen = tf.random.Generator.from_seed(42)

# Fresh, repeating real-data iterator (don’t reuse AE iterator)
warmup_real_ds  = make_dataset(X_timegan.astype(np.float32), parameters['batch_size'])
warmup_real_it  = iter(warmup_real_ds)

# Freeze supervisor during G warm-up (optional but recommended)
supervisor.trainable = False

# Weights (typical TimeGAN-style scaling)
lambda_s = 100.0   # supervised on generated latents
lambda_v = 100.0   # two-moment loss in data space

warmup_steps = 500  # 200–1000 is usually enough
g_losses_Shat, g_losses_V, g_losses_total = [], [], []

for step in range(warmup_steps):
    real_batch = next(warmup_real_it)
    real_batch = tf.cast(real_batch, tf.float32)

    Z = sample_noise(parameters['batch_size'], parameters['seq_len'], parameters['z_dim'])

    with tf.GradientTape() as tape:
        # Latent path from noise
        E_hat  = generator(Z, training=True)               # [B, T, hidden]
        H_hat  = supervisor(E_hat, training=False)         # [B, T, hidden]  (S frozen)
        X_hat  = recovery(H_hat, training=True)            # [B, T, dim]

        # Supervised dynamics on generated latent (teacher forcing on itself)
        H_hat_in  = H_hat[:, :-1, :]
        H_hat_tgt = H_hat[:,  1:, :]
        H_hat_pred = supervisor(H_hat_in, training=False)  # S frozen
        G_loss_S_hat = tf.reduce_mean(tf.square(H_hat_tgt - H_hat_pred))

 

        G_loss_V = moment_loss(real_batch, X_hat, y_idx)

        # Total warm-up loss (no adversarial yet)
        G_total = lambda_s * G_loss_S_hat + lambda_v * G_loss_V


    # Update ONLY the generator
    g_vars = generator.trainable_variables
    g_grads = tape.gradient(G_total, g_vars)
    generator_optimizer.apply_gradients(zip(g_grads, g_vars))

    # Logging buffers
    g_losses_Shat.append(float(G_loss_S_hat))
    g_losses_V.append(float(G_loss_V))
    g_losses_total.append(float(G_total))

    if step % 100 == 0:
        print(f"Warmup {step}/{warmup_steps} | "
              f"G_S_hat: {np.mean(g_losses_Shat[-100:]):.6f} | "
              f"G_V: {np.mean(g_losses_V[-100:]):.6f} | "
              f"G_total: {np.mean(g_losses_total[-100:]):.6f}")

# (optional) unfreeze supervisor for joint training
supervisor.trainable = True


In [ ]:
# Set once at top-level (because your D returns logits)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

@tf.function
def train_generator_step(real_batch):
    """
    Joint-phase generator step that matches the original TimeGAN behavior.
    - Updates G + S together (as in G_solver optimizing g_vars + s_vars).
    - Uses supervised loss on REAL H (S(H) vs H), like the original.
    - Includes the two adversarial terms (on H_hat and on E_hat) and the moment loss.
    """
    real_batch = tf.cast(real_batch, tf.float32)

    # Fresh noise every step (you already defined sample_noise)
    Z_mb = sample_noise(parameters['batch_size'],
                        parameters['seq_len'],
                        parameters['z_dim'])

    with tf.GradientTape() as tape:
        # ---- Fake path (depends on G, so grads flow to G & S) ----
        E_hat = generator(Z_mb, training=True)            # [B, T, hidden]
        H_hat = supervisor(E_hat, training=True)          # [B, T, hidden]
        #X_hat = recovery(H_hat, training=True)            # [B, T, dim]   ####if i want to return to before
        X_hat = recovery(H_hat, training=False)

        
        # Discriminator predictions on fakes
        Y_fake   = discriminator(H_hat, training=True)    # S(G(Z))
        Y_fake_e = discriminator(E_hat, training=True)    # G(Z) pre-supervisor

        # 1) Adversarial terms
        g_loss_u   = bce(tf.ones_like(Y_fake),   Y_fake)
        g_loss_u_e = bce(tf.ones_like(Y_fake_e), Y_fake_e)

        # 2) Supervised term on REAL latent (keeps S aligned to real H)
        H = embedder(real_batch, training=False)          # [B, T, hidden]
        H_hat_supervise = supervisor(H, training=True)    # S(H)
        g_loss_s = tf.reduce_mean(tf.square(H[:, 1:, :] - H_hat_supervise[:, :-1, :]))

        # 3) Two-moment loss in data space
        g_loss_v = moment_loss(real_batch, X_hat ,y_idx)

        # ---- Total (original-style weights) ----
        gamma = parameters.get('gamma', 1.0)  # weight for the G(Z) adversarial path
        G_total = g_loss_u + gamma * g_loss_u_e \
                  + 100.0 * tf.sqrt(g_loss_s + 1e-8) \
                  + 100.0 * g_loss_v

    # Update G + S together (like original G_solver over g_vars + s_vars)
    vars_to_update = generator.trainable_variables + supervisor.trainable_variables
    grads = tape.gradient(G_total, vars_to_update)
    generator_optimizer.apply_gradients(zip(grads, vars_to_update))

    # Return components for logging
    return (G_total, g_loss_u, g_loss_u_e, g_loss_s, g_loss_v)



# after (recommended for stability during G step)
X_hat = recovery(H_hat, training=False)

In [ ]:
#adding more condition to prevent d from loosing power
FREEZE_STEPS = 1000              # e.g., 500–1000
embedder.trainable = False       # freeze E for warmup

print('Start Joint Training')

iterations   = parameters.get('joint_iterations', 3000)
log_interval = parameters.get('log_interval', 100)
disc_thresh  = 1.8

# Iterator over real data
joint_dataset = make_dataset(X_timegan.astype(np.float32), parameters['batch_size'])
joint_iter    = iter(joint_dataset)

# For logging
step_g_loss_u = step_g_loss_u_e = step_g_loss_s = step_g_loss_v = 0.0
step_e_loss_t0 = 0.0
step_d_loss = 0.0

for itt in range(iterations):
    # --- TTUR nudge for D (first ~500 iters) ---
   # top of loop
    if itt < 500:
      lr_D = 4e-4   # brief boost (already in place)
    elif itt < 1500:
      lr_D = 2e-4   # COOL DOWN
    else:
      lr_D = 1e-4   # steady-low
    try:
      discriminator_optimizer.learning_rate.assign(lr_D)
    except Exception:
      import tensorflow.keras.backend as K
      K.set_value(discriminator_optimizer.learning_rate, lr_D)

    # --- Unfreeze embedder exactly at FREEZE_STEPS ---
    if (itt == FREEZE_STEPS) and (embedder.trainable is False):
        embedder.trainable = True

    # ---- (Generator + (optional) Embedder) x 2 ----
    for _ in range(2):
        # Generator step (updates G+S inside)
        x_batch = next(joint_iter)
        G_total, step_g_loss_u, step_g_loss_u_e, step_g_loss_s, step_g_loss_v = train_generator_step(x_batch)

        # Embedder step only after unfreeze
        if embedder.trainable:
            x_batch_e = next(joint_iter)
            step_e_loss_t0 = train_embedder_step(x_batch_e, y_idx)  # returns recon MSE

    # ---- Discriminator: probe + gated updates ----
    x_batch_d = next(joint_iter)
    d_check = float(probe_discriminator_loss_min(x_batch_d))

    # If D is clearly off, do an extra D step ONCE in a while (your modulo-20 rule)
    if ((d_check > 2.0) or (float(step_g_loss_u) > 3.0)) and (itt % 50 == 0):
        step_d_loss, *_ = train_discriminator_step(x_batch_d)
        x_batch_d2 = next(joint_iter)                  # fresh batch
        step_d_loss, *_ = train_discriminator_step(x_batch_d2)
    else:
        # Normal gate
        if d_check > disc_thresh:
            step_d_loss, *_ = train_discriminator_step(x_batch_d)
        else:
            step_d_loss = d_check  # just log the probe

    # ---- Logging (reference style) ----
    if itt % log_interval == 0:
        print('step: ' + str(itt) + '/' + str(iterations) +
              ', d_loss: '   + f'{float(step_d_loss):.4f}' +
              ', g_loss_u: ' + f'{float(step_g_loss_u):.4f}' +
              ', g_loss_s: ' + f'{np.sqrt(float(step_g_loss_s)+1e-8):.4f}' +
              ', g_loss_v: ' + f'{float(step_g_loss_v):.4f}' +
              ', e_loss_t0: '+ f'{np.sqrt(float(step_e_loss_t0)+1e-8):.4f}')

print('Finish Joint Training')

